In [1]:
import pandas as pd
import numpy as np


In [ ]:
df1=pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv',low_memory=False)
df2=pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv',low_memory=False)
df3=pd.read_csv('/kaggle/input/the-movies-dataset/links.csv',low_memory=False)
#df4=pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv',low_memory=False)
df5=pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv',low_memory=False)
df6=pd.read_csv('/kaggle/input/the-movies-dataset/ratings.csv',low_memory=False)
#df7=pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv',low_memory=False)


In [ ]:
# Remove rows with non-integer 'id'
df5 = df5[df5['id'].str.isnumeric()]

# Now convert the 'id' column to integers
df5['id'] = df5['id'].astype('int64')


In [ ]:
# Ensure id columns are of the same type
df1['id'] = df1['id'].astype(int)
df2['id'] = df2['id'].astype(int)
df5['id'] = df5['id'].astype(int)

# Merge the dataframes
merged_df = df1.merge(df2, on='id', how='left').merge(df5, on='id', how='left')


In [ ]:
merged_df

In [ ]:
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity




In [ ]:

# Function to join names from list of dictionaries
def join_names(json_like_str):
    try:
        # Convert the string representation of list of dicts to actual list of dicts
        list_dicts = ast.literal_eval(json_like_str)
        return ' '.join([d['name'] for d in list_dicts if 'name' in d])
    except:
        # In case of any error, return an empty string
        return ''

# Apply join_names to 'genres' and 'keywords' in merged_df
merged_df['genres'] = merged_df['genres'].apply(join_names)
merged_df['keywords'] = merged_df['keywords'].apply(join_names)

# Combine 'genres', 'keywords', and 'overview' into 'combined_features'
merged_df['combined_features'] = merged_df['genres'] + " " + merged_df['keywords'] + " " + merged_df['overview']

# Fill NaN values in 'combined_features'
merged_df['combined_features'] = merged_df['combined_features'].fillna('')


In [ ]:
# Initialize and fit the TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=10000, dtype=np.float32)
tfidf_matrix = tfidf.fit_transform(merged_df['combined_features'])

# Function to get movie recommendations
def get_recommendations(title, description):
    user_input = title + " " + description
    user_input_tfidf = tfidf.transform([user_input])
    cosine_sim = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()
    sim_scores = sorted(list(enumerate(cosine_sim)), key=lambda x: x[1], reverse=True)[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return merged_df['title'].iloc[movie_indices]


In [ ]:
# Example usage
recommended_movies = get_recommendations('Toy Story', 'A story about friendship and adventure')
print(recommended_movies)

In [ ]:
#CREDITS KEYWORDS AND METADATA HAS BEEN USED TO MERGE AND MAKE A NEW COMBINED DATA_FRAME